In [13]:
from bs4 import BeautifulSoup
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import win32gui, win32con
import base64
import sqlite3
import win32crypt
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import requests as rq
import re
import datetime
import pymysql
import time

In [14]:
#
url = "https://web.pcc.gov.tw/tps/pss/tender.do?method=goSearch&searchMode=common&searchType=basic"
url_head = "https://web.pcc.gov.tw/tps/pss"
response = rq.get(url,cookies={'over18': '1'})
html_doc = response.text
#解析表單的 option 值，用以放到 post api 的 tenderWay & tenderType欄位
soup = BeautifulSoup(html_doc, 'lxml')

pages_option_type_html=soup.select('select#tenderType option')
pages_option_type=[]
for i in pages_option_type_html:
    pages_option_type.append(i.get('value'))
print(pages_option_type)
# ['tenderDeclaration' 招標公告, 'searchAppeal' 公開徵求, 'publicRead' 公開閱覽, 'predict' 政府採購預告]

pages_option_way=[]
for i in soup.select('select#tenderWay option'):
    try:
        int(i.get('value'))
    except:
        pages_option_way_all = i.get('value')
        continue
    pages_option_way.append({'code': i.get('value'),'text': i.text.strip()})
print(pages_option_way_all)


['tenderDeclaration', 'searchAppeal', 'publicRead', 'predict']
1,2,3,4,5,6,7,10,12


In [15]:
def GetString(LocalState):
    with open(LocalState,'r',encoding='utf-8') as f:
        s=json.load(f)['os_crypt']['encrypted_key']
    return s

def pull_the_key(base64_encrypted_key):
    encrypted_key_with_header=base64.b64decode(base64_encrypted_key)
    encrypted_key=encrypted_key_with_header[5:]
    key=win32crypt.CryptUnprotectData(encrypted_key,None,None,None,0)[1]
    return key

def DecryptString(key,data):
    nonce,cipherbytes=data[3:15],data[15:]
    aesgcm=AESGCM(key)
    plainbytes=aesgcm.decrypt(nonce,cipherbytes,None)
    plaintext=plainbytes.decode('utf-8')
    return plaintext

def test_cookies():
    LocalState = r'C:\Users\User\AppData\Local\Google\Chrome\User Data\Local State' #密钥文件
    Cookies = r'C:\Users\User\AppData\Local\Google\Chrome\User Data\Default\Network\Cookies' #cookie文件
    
    con = sqlite3.connect(Cookies)
    res_cookie = con.execute('select host_key,name,encrypted_value from cookies where host_key like "%web.pcc.gov.tw%"').fetchall()
    key = pull_the_key(GetString(LocalState))        
    con.close()

#    字典回傳  
    datas={}
    for i in res_cookie:
        datas[i[1]]=DecryptString(key, i[2])
    return datas
cookie_data = test_cookies()

In [192]:
# tenderType == 'tenderDeclaration' 招標公告
    # form_data = {'method':'search','searchMethod':'true','tenderUpdate':'','searchTarget':'','orgName':'','orgId':'','hid_1':'1','tenderName':'','tenderId':'','tenderType':'tenderDeclaration','tenderWay':pages_option_way_all,'tenderDateRadio':'on','tenderStartDateStr':'111/01/06','tenderEndDateStr':'111/01/12','tenderStartDate':'111/01/12','tenderEndDate':'111/01/12','isSpdt':'N','proctrgCate':'','btnQuery':'查詢','hadUpdated':''}
    # r = rq.post('https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic', data = form_data)
url_tenderDeclaration = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex="
url_searchAppeal = "https://web.pcc.gov.tw/tps/tps/tp/main/tps/tp/searchListAppealVendorCommon.do?__PageBase=0&__PageIndex="
url_publicRead = "https://web.pcc.gov.tw/tps/tps/tp/main/pms/tps/tp/commonPublicReadFormListPageControl.do?__PageBase=0&__PageIndex="
url_predict = ""

def gov_header_crawler(url,url_type):
    raw_datas=[]
    for ii in range(1,100):
        r = rq.get(url + str(ii), cookies = cookie_data)
        print(r.url)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'lxml')
        
        if url_type == "tenderDeclaration":
            datarow_of_end = -1
            count_per_page = 100
            data = soup.select('div#print_area table tr')
            for i in data[1:datarow_of_end]:
                #項次
                no=i.select('td')[0].text.strip()
                #機關名稱
                proposer_name=i.select('td')[1].text.strip()
                #標案案號
                bid_no=re.match(r'(.*)\r\n\t', i.select('td')[2].text.strip(), flags=0).group(1)
                # 是否更正
                if i.select('td')[2].select('font'):
                    ischange=1
                else:
                    ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[2].select('a')[0].text.strip()
                # 傳輸次數
                times=i.select('td')[3].text.strip()
                # 招標方式
                typ=i.select('td')[4].text.strip()
                # 採購性質
                clas=i.select('td')[5].text.strip()
                # 公告日期
                date=i.select('td')[6].text.strip()
                # 截止投標
                enddate=i.select('td')[7].text.strip()
                # 預算金額
                budget=i.select('td')[8].text.strip()
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange})
        elif url_type == "searchAppeal":
            datarow_of_end = -1
            count_per_page = 10
            data = soup.select('center table:nth-child(1) tr')
            for i in data[17:datarow_of_end]:
                #項次
                no=i.select('td')[0].text.strip()
                #機關名稱
                proposer_name=i.select('td')[1].text.strip()
                #標案案號
                bid_no=i.select('td')[2].text.strip()
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[3].text.strip()
                # 傳輸次數 (公告次數)
                times=i.select('td')[4].text.strip()
                # 招標方式
                typ=""
                # 採購性質
                clas=""
                # 公告日期
                date=re.match(r'(.*)\s*－\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(1).strip()
                # 截止投標
                enddate=re.match(r'(.*)\s*－\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(2).strip()
                # 預算金額
                budget=""
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange})
        elif url_type == "publicRead":
            datarow_of_end = -2
            count_per_page = 10
            data = soup.select('td#page table:nth-child(3) tr')
            for i in data[15:datarow_of_end]:
                #項次
                no=i.select('td')[0].text.strip()
                #機關名稱
                proposer_name=i.select('td')[1].text.strip()
                #標案案號
                bid_no=i.select('td')[2].text.strip()
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[3].text.strip()
                # 傳輸次數 (公告次數)
                times=i.select('td')[4].text.strip()
                # 招標方式
                typ=""
                # 採購性質
                clas=""
                # 公告日期
                date=re.match(r'(.*)\s*─\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(1).strip()
                # 截止投標
                enddate=re.match(r'(.*)\s*─\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(2).strip()
                # 預算金額
                budget=""
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange})
        elif url_type == "predict" :  
            return
        else:
            return 
        all_data = int(data[datarow_of_end].select('span')[-1].text)
        print(all_data)
        print(ii*count_per_page)
        if ii*count_per_page > all_data:
            break
#         next_page = data[-1].select('a')[-2].get('href')
#         url = url_head + next_page[1:]
#         response = rq.get(url, cookies = cookie_data)
#         html_doc = response.text
        
    return raw_datas

In [194]:
x1 = gov_header_crawler(url_tenderDeclaration,"tenderDeclaration")
x2 = gov_header_crawler(url_searchAppeal,"searchAppeal")
x3 = gov_header_crawler(url_publicRead,"publicRead")

https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=1
1152
100
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=2
1152
200
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=3
1152
300
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=4
1152
400
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=5
1152
500
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=6
1152
600
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=7
1152
700
https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=8
1152
800
https://web.pcc.gov.tw/tps/pss/tender.do?searchM

In [198]:
r = rq.get(url_publicRead + str(1), cookies = cookie_data)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'lxml')
data = soup.select('td#page table:nth-child(3) tr')
#print(data[-1])
for i in data[15:-2]:
    #print(i)
#     print(i.select('td')[0].text.strip())
#     print(i.select('td')[1].text.strip())
#     print(i.select('td')[2].text.strip())
#     print(i.select('td')[3].text.strip())
#     print(i.select('td')[4].text.strip())
    print(i.select('td')[6])
    ss=i.select('td')[5].text.strip()
    print('--')

<td>
<button class="T11c" id="btn50027576" onclick="verifyUploadFile('/tps/tps/tom/main/tps/tom/obtainment.do','init','50027576','3')" style="display:none;" type="button">公開閱覽</button>
<script language="javascript">
								  	document.getElementById("btn50027576").style.display="block";
								</script>
</td>
--
<td>
<button class="T11c" id="btn50027407" onclick="verifyUploadFile('/tps/tps/tom/main/tps/tom/obtainment.do','init','50027407','3')" style="display:none;" type="button">公開閱覽</button>
<script language="javascript">
								  	document.getElementById("btn50027407").style.display="block";
								</script>
</td>
--
<td>
<button class="T11c" id="btn50027574" onclick="verifyUploadFile('/tps/tps/tom/main/tps/tom/obtainment.do','init','50027574','3')" style="display:none;" type="button">公開閱覽</button>
<script language="javascript">
								  	document.getElementById("btn50027574").style.display="block";
								</script>
</td>
--
<td>
<button class="T11c" id="btn50027577" onclick="

In [19]:
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "As123459362",
    "db": "pttcrawler",
    "charset": "utf8",
    "autocommit":True
}

def send_db(raw_datas, db_settings):
# import charts
# 資料庫參數設定

    time_start = time.time() #開始計時
    wrong = 0
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id="select MAX(UID) FROM log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = "INSERT INTO gov_purchase(log_UID, proposer_name, bid_no, bid_name, times, typ, clas, date, enddate, budget, ischange)VALUES(%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)"
            command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始
            cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "start", "", ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 執行
            for raw_data in raw_datas:
                try:
                    cursor.execute(command, (int(log_UID), raw_data["proposer_name"], raw_data["bid_no"], raw_data["bid_name"], raw_data["times"], raw_data["typ"], raw_data["clas"], raw_data["date"], raw_data["enddate"], raw_data["budget"], raw_data["ischange"]))
                except Exception as err:
                    wrong = wrong + 1
                    cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "wrong", str(raw_data), str(err)))
                    continue
            # 紀錄結束
            if wrong==0:
                cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "finish", "well_completed", ""))
            else:
                cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "finish", wrong + "records are wrong", ""))
    except Exception as ex:
        print(ex)

    time_end = time.time()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('time cost', time_c, 's')

In [195]:
send_db(x1, db_settings)
send_db(x2, db_settings)
send_db(x3, db_settings)

time cost 1.8056399822235107 s
time cost 0.8487701416015625 s
time cost 0.033029794692993164 s


In [21]:
len(raw_datas)

NameError: name 'raw_datas' is not defined